# Raw IntegraSUS processing

**Author**: Higor S. Monteiro<br>
**Date**: 2022/04/04

Processing of the raw IntegraSUS to serve as complement of the 'Vacine Já' database.

The goal here is to create an automatic routine to generate a new dataset from the raw IntegraSUS: Dataset containing individuals per row with their cumulative metadata on Covid-19 tests. This data will help us to define a population for the tested individuals for the city of Fortaleza. 

## Lib

In [1]:
import sys
sys.path.append("..")

import os
import csv

import numpy as np
import pandas as pd
import lib.utils as utils
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

## Set paths and load data

In [2]:
base_path = os.path.join(os.environ["USERPROFILE"], "Documents")
data_folder = os.path.join(base_path, "data")
tests_folder = os.path.join(data_folder, "testes_covid19")

project_folder = os.path.join(base_path, "projects", "vaccine-eff-fortaleza")
schema_folder = os.path.join(project_folder, "output", "data")

### Read raw Integrasus through `csv` package

In [3]:
def integra_csv(fname, delimiter, encoding, usecols=None, nrows=None):
    '''
    
    '''
    cols = None
    # --> extract column names
    col_hash = dict()
    with open(fname, encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter, quoting=csv.QUOTE_NONE)
        for row in csv_reader:
            cols = row
            break
    cols_all = cols
    [ col_hash.update({colname: n}) for n, colname in enumerate(cols_all) ]
    
    # --> Initiate data structure
    if usecols is not None:
        cols = usecols 
    data_hash = dict()
    for key in cols:
        data_hash.update({key: []})
        
    irregular_records = []
    # --> Load file
    with open(fname, encoding=encoding) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=delimiter, quoting=csv.QUOTE_NONE)
        line_count = 0
        for row in csv_reader:
            if len(row)!=len(cols_all):
                irregular_records.append(row)
                continue
            # If number of rows is provided
            if nrows is not None and line_count==nrows:
                break
            # First line containing the headers
            if line_count==0:
                line_count += 1
            # Process the data
            else:
                # ...
                [ data_hash[j].append(row[col_hash[j]]) for j in usecols]
                line_count += 1
                
    results = {"ALL COLUMNS": cols_all, "DATA": pd.DataFrame(data_hash), "IRREGULAR RECORDS": irregular_records}
    return results

### Read records for selected fields

In [71]:
selected_cols = ["id", "nome_paciente", "nome_mae", "cpf", "cns", "data_nascimento", "idade_paciente", "sexo_paciente", "codigo_paciente", 
                 "bairro_paciente", "codigo_municipio_paciente", "municipio_paciente", "bairro_paciente_geocoder", "data_solicitacao_exame", 
                 "data_coleta_exame", "data_resultado_exame", "resultado_final_exame", "data_obito", "data_inicio_sintomas", "cep_paciente",
                 "id_sivep", "data_internacao_sivep", "data_entrada_uti_sivep", "data_saida_uti_sivep"]

fname = os.path.join(tests_folder, "base-dados-integrasus-2022-04-04-07-18-51.csv") 
results = integra_csv(fname, delimiter=";", encoding="utf-8", nrows=800000, usecols=selected_cols)

df = results["DATA"]
fort_df = df[df["municipio_paciente"]=="FORTALEZA"]

In [72]:
fort_df.shape

(279648, 24)

## Transform data

In [73]:
fort_df["cpf"] = fort_df["cpf"].apply(lambda x: x if pd.notna(x) and len(x)>0 else np.nan)
fort_df["cpf"] = fort_df["cpf"].apply(lambda x: f"{float(x):11.0f}".replace(" ", "0") if pd.notna(x) else np.nan)

#fort_df['nome_mae'] = fort_df['nome_mae'].apply(lambda x: x if len(x)>0 and pd.notna(x) else np.nan)
#fort_df["nome tratado"] = fort_df["nome_paciente"].apply(lambda x: utils.replace_string(x) if pd.notna(x) else np.nan)
#fort_df["nome hashcode"] = fort_df["nome_paciente"].apply(lambda x: utils.replace_string_hash(x) if pd.notna(x) else np.nan)
#fort_df["nome mae tratado"] = fort_df["nome_mae"].apply(lambda x: utils.replace_string(x) if pd.notna(x) else np.nan)
#fort_df["nome mae hashcode"] =fort_df["nome_mae"].apply(lambda x: utils.replace_string_hash(x) if pd.notna(x) else np.nan)

fort_df["data_nascimento"] = pd.to_datetime(fort_df["data_nascimento"], format="%Y/%m/%d", errors="coerce")
fort_df["data_coleta_exame"] = pd.to_datetime(fort_df["data_coleta_exame"], format="%Y/%m/%d", errors="coerce")
fort_df["data_inicio_sintomas"] = pd.to_datetime(fort_df["data_inicio_sintomas"], format="%Y/%m/%d", errors="coerce")
fort_df["data_internacao_sivep"] = pd.to_datetime(fort_df["data_internacao_sivep"], format="%Y/%m/%d", errors="coerce")
fort_df["data_entrada_uti_sivep"] = pd.to_datetime(fort_df["data_entrada_uti_sivep"], format="%Y/%m/%d", errors="coerce")
fort_df["data_saida_uti_sivep"] = pd.to_datetime(fort_df["data_saida_uti_sivep"], format="%Y/%m/%d", errors="coerce")
fort_df["data_evolucao_caso_sivep"] = pd.to_datetime(fort_df["data_saida_uti_sivep"], format="%Y/%m/%d", errors="coerce")
fort_df["data_obito"] = pd.to_datetime(fort_df["data_obito"], format="%Y/%m/%d", errors="coerce")
fort_df["data_resultado_exame"] = pd.to_datetime(fort_df["data_resultado_exame"], format="%Y/%m/%d", errors="coerce")
fort_df["data_solicitacao_exame"] = pd.to_datetime(fort_df["data_solicitacao_exame"], format="%Y/%m/%d", errors="coerce")

fort_df["sexo_paciente"] = fort_df["sexo_paciente"].map({"FEMININO": "F", "MASCULINO": "M", 
                                                         "INDEFINIDO": "I", "IGNORADO": "I", "": np.nan})

In [74]:
people = fort_df.astype(str).groupby("codigo_paciente").agg(";".join).reset_index()

In [75]:
people["id"] = people["id"].apply(lambda x: [int(cid) for cid in x.split(";")] )
people["nome_paciente"] = people["nome_paciente"].apply(lambda x: [name for name in x.split(";") if name!="nan"][0] if len([name for name in x.split(";") if name!="nan"])>0 else np.nan )
people["cpf"] = people["cpf"].apply(lambda x: [cpf for cpf in x.split(";") if cpf!="nan"][0] if len([cpf for cpf in x.split(";") if cpf!="nan"])>0 else np.nan)
people["data_nascimento"] = people["data_nascimento"].apply(lambda x: [cpf for cpf in x.split(";") if cpf!="nan"][0] if len([cpf for cpf in x.split(";") if cpf!="nan"])>0 else np.nan)
people["idade_paciente"] = people["idade_paciente"].apply(lambda x: [age for age in x.split(";") if age!="nan"][0] if len([age for age in x.split(";") if age!="nan"])>0 else np.nan)
people["sexo_paciente"] = people["sexo_paciente"].apply(lambda x: [age for age in x.split(";") if age!="nan"][0] if len([age for age in x.split(";") if age!="nan"])>0 else np.nan)

people["data_coleta_exame"] = people["data_coleta_exame"].apply(lambda x: [pd.to_datetime(cid) for cid in x.split(";") if cid!="nan"] )
people["data_solicitacao_exame"] = people["data_solicitacao_exame"].apply(lambda x: [pd.to_datetime(cid) for cid in x.split(";") if cid!="nan"] )
people["resultado_final_exame"] = people["resultado_final_exame"].apply(lambda x: [cid for cid in x.split(";") if cid!="nan"] )
people["data_resultado_exame"] = people["data_resultado_exame"].apply(lambda x: [pd.to_datetime(cid) for cid in x.split(";") if cid!="nan"] )
people["data_obito"] = people["data_obito"].apply(lambda x: [pd.to_datetime(cid) for cid in x.split(";") if cid!="nan"] )
people["data_inicio_sintomas"] = people["data_inicio_sintomas"].apply(lambda x: [pd.to_datetime(cid) for cid in x.split(";") if cid!="nan"] )

In [80]:
people.iloc[:10,12:20]

,bairro_paciente_geocoder,data_solicitacao_exame,data_coleta_exame,data_resultado_exame,resultado_final_exame,data_obito,data_inicio_sintomas,cep_paciente
0,,[2021-05-21 00:00:00],[2021-05-21 00:00:00],[2021-05-23 02:53:07],[Negativo],[NaT],[NaT],
1,;,"[2020-11-26 00:00:00, 2020-11-26 00:00:00]","[2020-11-26 00:00:00, 2020-11-26 00:00:00]","[2020-11-29 15:19:02, 2020-11-29 15:19:02]","[Positivo, Positivo]","[NaT, NaT]","[2020-11-23 00:00:00, 2020-11-23 00:00:00]",60000000;60000000
2,,[2021-05-13 00:00:00],[2021-05-13 00:00:00],[2021-05-18 09:13:43],[Negativo],[NaT],[2021-05-07 00:00:00],60870180
3,,[NaT],[2020-06-24 00:00:00],[2020-06-26 00:00:00],[Negativo],[NaT],[NaT],
4,,[2022-01-08 00:00:00],[2022-01-08 00:00:00],[2022-01-11 22:11:54],[Negativo],[NaT],[NaT],
5,,[NaT],[2021-06-12 00:00:00],[2021-06-12 00:00:00],[Negativo],[NaT],[2021-06-05 00:00:00],00000000
6,;,"[NaT, NaT]","[2021-01-08 00:00:00, 2021-05-16 00:00:00]","[2020-01-11 00:00:00, 2021-09-15 00:00:00]","[Negativo, Negativo]","[NaT, NaT]","[2021-01-05 00:00:00, NaT]",60120000;
7,;;,"[NaT, NaT, NaT]","[2020-06-12 00:00:00, 2021-01-06 00:00:00, 202...","[2020-06-22 00:00:00, 2021-01-07 00:00:00, 202...","[Negativo, Positivo, Negativo]","[NaT, NaT, NaT]","[NaT, NaT, NaT]",;;
8,;,"[2022-01-06 00:00:00, NaT]","[2022-01-06 00:00:00, 2022-01-14 00:00:00]","[2022-01-07 12:28:27, 2022-01-18 00:00:00]","[Negativo, Negativo]","[NaT, NaT]","[NaT, NaT]",60810720;
9,;,"[NaT, 2021-12-07 00:00:00]","[NaT, 2021-12-07 00:00:00]","[NaT, 2021-12-08 14:55:15]","[Provável, Negativo]","[NaT, NaT]","[2022-01-12 00:00:00, NaT]",60337350;


In [76]:
people[15:35]

,codigo_paciente,id,nome_paciente,nome_mae,cpf,cns,data_nascimento,idade_paciente,sexo_paciente,bairro_paciente,...,data_resultado_exame,resultado_final_exame,data_obito,data_inicio_sintomas,cep_paciente,id_sivep,data_internacao_sivep,data_entrada_uti_sivep,data_saida_uti_sivep,data_evolucao_caso_sivep
15,5e680df10e70f09a0d69643619da72c2,"[1652389, 1652390, 1652391]",JOSE ROSEMIRO DE MELO GONCALVES FILHO,;;,78407370304,;;,1976-11-12 00:00:00,43,M,AMADEU FURTADO;AMADEU FURTADO;,...,"[2020-06-18 00:00:00, NaT, 2020-06-18 00:00:00]","[Negativo, Provável, Negativo]","[NaT, NaT, NaT]","[2020-06-12 00:00:00, 2020-01-05 00:00:00, NaT]",60455600;60455600;,;;,NaT;NaT;NaT,NaT;NaT;NaT,NaT;NaT;NaT,NaT;NaT;NaT
16,5e681272abd06393585ee9064f2df475,[1652393],FRANCISCO ANTONIO DE OLIVEIRA CASIMIRO,,07620311319,,1998-02-21 00:00:00,23,M,JANGURUSSU,...,[2022-01-30 23:10:59],[Negativo],[NaT],[2022-01-24 00:00:00],00000000,,NaT,NaT,NaT,NaT
17,5e683196bd7060272af14b150da35a21,"[1652401, 1652402]",RENATO LIMA ALEXANDRE,;,04140698381,;,1992-02-19 00:00:00,29,M,BARRA DO CEARÁ;BARRA DO CEARÁ,...,"[2021-03-22 00:00:00, 2020-11-12 00:00:00]","[Negativo, Negativo]","[NaT, NaT]","[2021-03-16 00:00:00, 2020-11-12 00:00:00]",60332660;60332660,;,NaT;NaT,NaT;NaT,NaT;NaT,NaT;NaT
18,5e685af7e49ecfa11ea55a3c674d2dbc,[1652410],JANAINA MACIEL ALVES,,61464864349,,1979-12-17 00:00:00,40,F,MONTE CASTELO,...,[NaT],[Provável],[NaT],[2020-06-02 00:00:00],60320735,,NaT,NaT,NaT,NaT
19,5e686256414066060a38aea25a739f29,[1652413],JOAO MIGUEL SOUSA RAMOS,MARIA DA PAZ SOUSA LOPES,NaN,,2016-06-22 00:00:00,4,M,DIAS MACEDO,...,[2021-02-27 01:27:26],[Negativo],[NaT],[2021-02-25 00:00:00],60000000,,NaT,NaT,NaT,NaT
20,5e686c0d5e90459fde0c050ee2eaf0b5,[1652414],SEVERINO ANTONIO DE LIMA,,13522798864,,1970-09-04 00:00:00,50,M,MONDUBIM,...,[2021-05-11 10:51:29],[Positivo],[NaT],[2021-04-27 00:00:00],60000000,,NaT,NaT,NaT,NaT
21,5e6873dfa6a2a014c2fcc3062b90a7dc,"[1652415, 1652416]",MARIA NEMAURA FERREIRA,;,68129858304,;,1944-10-07 00:00:00,76,F,;MARECHAL RONDON (JUREMA),...,"[2021-02-04 04:26:01, 2021-01-31 00:00:00]","[Negativo, Negativo]","[NaT, NaT]","[NaT, 2021-01-29 00:00:00]",;61652640,;,NaT;NaT,NaT;NaT,NaT;NaT,NaT;NaT
22,5e68768a5a7e529f54a157c6cdc50767,[1652419],AMELIA CRISTINA GARCIA MENEZES MONCAO,MARIA DO SOCORRO MENESES GARCIA,13559079349,704009310244868,1957-09-22 00:00:00,63,F,ALDEOTA,...,[2020-12-08 04:50:57],[Negativo],[NaT],[NaT],,,NaT,NaT,NaT,NaT
23,5e6884ace5dc00fd1ae2027548fd5d14,[1652422],VALDIR DE SOUSA GOMES,FRANCISCA DAS CHAGAS DE SOUSA GOMES,46902457387,705003815236555,1967-10-16 00:00:00,54,M,BARRA DO CEARA,...,[NaT],[Provável],[NaT],[2022-01-14 00:00:00],60332650,,NaT,NaT,NaT,NaT
24,5e688f832292aee29df0ea92c370c3da,[1652423],ADRIANO MARTINS MOREIRA,,82939993300,,1979-03-10 00:00:00,42,M,MARAPONGA,...,[2022-01-13 00:00:00],[Positivo],[NaT],[2022-01-08 00:00:00],60711035,,NaT,NaT,NaT,NaT
